## 使用Python实现GRPC客户端代码

### 生成protobuf 和 grpc 的 pb.py 文件

In [ ]:
import os
from grpc_tools import protoc

proto_file = "protos/message.proto"
output_dir = "../build/src/protos"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

protoc.main((
    '',
    f'-I{os.path.dirname(proto_file)}',
    f'--python_out={output_dir}',
    f'--grpc_python_out={output_dir}',
    proto_file,
))

In [ ]:
from __future__ import print_function
import grpc

import sys
import os

# 动态添加路径到 sys.path
proto_path = os.path.abspath(os.path.join('..', 'build', 'src', 'protos'))
sys.path.append(proto_path)

# 现在可以导入 message_pb2
import message_pb2
import message_pb2_grpc



In [ ]:
import threading
import time

# 定义要发送的请求
def send_request(agent_name):
    # 每个线程发送1000次消息，间隔 10ms
    response_messages = []
    for req_time in range(1000):
        with grpc.insecure_channel('localhost:50051') as channel:
            stub = message_pb2_grpc.GreetServiceStub(channel)
            response = stub.SayHello(message_pb2.HelloRequest(name=agent_name))
            response_messages.append(response.message)
        time.sleep(0.01)

    print(f"{agent_name} Stoped! received messages: {response_messages}")


def run():
    # 创建100个线程并启动
    threads = []
    for agent_idx in range(100):
        agent_name = f"Agent {agent_idx}"
        thread = threading.Thread(target=send_request, args=(agent_name,))
        thread.start()
        threads.append(thread)
        print(f"Greeter client started! Name = {agent_name}")
        time.sleep(0.05)  # Wait for 50 milliseconds

    # 等待所有线程完成
    for thread in threads:
        thread.join()


def main():
    run()


if __name__ == "__main__":
    main()